<a href="https://colab.research.google.com/github/aavam/GEN_AI_PROMPT/blob/main/AHS_Learner_Notebook_final_project_GEN_AI_MSFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project : A Case Study of ExpressWay Logistics**

**Business Overview:**

ExpressWay Logistics is a dynamic logistics service provider, committed to delivering efficient, reliable and cost-effective courier transportation and warehousing solutions. With a focus on speed, precision and customer satisfaction, we aim to be the go-to partner for our customers seeking seamless courier services. Our core service involves ensuring operational efficiency throughout our delivery and courier services, including inventory management, durable packaging and swift dispatch of couriers, real time tracking of shipments and on-time delivery of couriers as promised. We are committed to enhance our logistics and courier services and improve seamless connectivity for our customers.

**Current Challenge:**

ExpressWay Logistics faces numerous challenges in ensuring seamless deliveries and customer satisfaction. These challenges include managing various customer demands simultaneously, addressing delays in deliveries and ensuring products arrive intact and safe. Additionally, the company struggles with complexity of efficiently storing and handling a large volume of packages and ultimately meeting customer expectations. Moreover, maintaining a skilled workforce capable of handling various aspects of logistics operations presents its own set of challenges. Overcoming these obstacles requires a comprehensive approach that integrates innovative technology, strategic planning, and continuous improvement initiatives to ensure smooth operations and exceptional service delivery.

**Objective:**

Our primary objective is to conduct a sentiment analysis of user-generated reviews across various digital channels and platforms. By paying attention to their feedback, we want to find ways to make our services better - like handling different customer demands simultaneously, dealing with late deliveries, and keeping packages secured and intact. Through the application of prompt engineering methodologies and sentiment analysis, we'll figure out if sentiments expressed by users for our courier services are Positive or Negative. This will help us understand where we need to improve in order to meet customer expectations and keep them happy. With a focus on getting better all the time, we'll overcome the challenges at ExpressWay Logistics and make our services the best.

**Data Description:**

The dataset titled "courier-service_reviews.csv" is structured to facilitate sentiment analysis for courier service reviews. Here's a brief description of the data columns:

1. id: This column contains unique identifiers for each review entry. It helps in distinguishing and referencing individual reviews.
2. review: This column includes the actual text of the courier service reviews. The reviews are likely composed of customer opinions and experiences regarding different aspects of the services provided by ExpressWay Logistics.
3. sentiment: This column provides an additional layer of classification (positive and negative) for the mentioned reviews.

##**Step 1. Setup (2 Marks)**

(A) Writing/Creating the config.json file  (2 Marks)

### Installation

In [ ]:
!pip install openai==1.2 tiktoken datasets session-info --quiet

### Imports

In [ ]:
# Import all Python packages required to access the Azure Open AI API.
# Import additional packages required to access datasets and create examples.

from openai import AzureOpenAI
import json
import random
import tiktoken
#import session_info

import pandas as pd
import numpy as np

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tabulate import tabulate

In [ ]:
#session_info.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Authentication

**(A) Writing/Creating the config.json file (2 Marks)**

In [ ]:
# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "05f8a427ba38465e93a7f386ff350c64",
    "AZURE_OPENAI_ENDPOINT": "https://test-openai-123456.openai.azure.com/",
    "AZURE_OPENAI_APIVERSION": "2024-05-01-preview",
    "CHATGPT_MODEL": "ahs-sep-setup"
}

In [ ]:
# Write the configuration information into the config.json file
with open('/content/drive/MyDrive/Colab_Notebooks/GEN-AI-Final-Project/config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

Config file created successfully!


In [ ]:
with open('/content/drive/MyDrive/Colab_Notebooks/GEN-AI-Final-Project/config.json', 'r') as az_creds:
    data = az_creds.read()

In [ ]:
creds = json.loads(data)

In [ ]:
client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [ ]:
chat_model_id = creds["CHATGPT_MODEL"]

In [ ]:
client.chat.completions.create(
    model=chat_model_id,
    messages = [
       {'role': 'user', 'content': 'Count to 10'}
    ]
)

ChatCompletion(id='chatcmpl-AAS5dfZ8BXiN9fD5wz7AJVtyq67wo', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='1, 2, 3, 4, 5, 6, 7, 8, 9, 10.', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, logprobs=None)], created=1727054621, model='gpt-35-turbo', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=29, prompt_tokens=12, total_tokens=41), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'sev

### Utilities

In [ ]:
def num_tokens_from_messages(messages):

    """
    Return the number of tokens used by a list of messages.
    Adapted from the Open AI cookbook token counter
    """

    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Each message is sandwiched with <|start|>role and <|end|>
    # Hence, messages look like: <|start|>system or user or assistant{message}<|end|>

    tokens_per_message = 3 # token1:<|start|>, token2:system(or user or assistant), token3:<|end|>

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

    return num_tokens

## Task : Sentiment Analysis

##**Step 2: Assemble Data (5 Marks)**

(A) Upload and Read csv File (2 Marks)

(B) Count Positive and Negative Sentiment Reviews (1 Marks)

(C) Split the Dataset (2 Marks)

**(A) Upload and read csv file (2 Marks)**

In [ ]:
 cs_reviews_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/GEN-AI-Final-Project/courier-service_reviews.csv')
# Read CSV File Here

In [ ]:
cs_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         131 non-null    int64 
 1   review     131 non-null    object
 2   sentiment  131 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.2+ KB


In [ ]:
cs_reviews_df.sample(5)

,id,review,sentiment
13,14,What sets ExpressWay Logistics apart from othe...,Positive
117,118,"When it comes to handling parcels, ExpressWay ...",Positive
118,119,I had a last-minute change to my shipping addr...,Positive
125,126,ExpressWay Logistics delivers more than just p...,Positive
63,64,ExpressWay Logistics promises efficient delive...,Negative


**(B) Count Positive and Negative Sentiment Reviews (1 Marks)**

In [ ]:
print("Sentiment")
print("Postive",sum(cs_reviews_df.sentiment == 'Positive'))
print("Negative",sum(cs_reviews_df.sentiment == 'Negative'))



Sentiment
Postive 68
Negative 63


In [ ]:
cs_reviews_df.shape

(131, 3)

**(C) Split the Dataset (2 Marks)**

In [ ]:
cs_reviews_df, cs_gold_examples_df = train_test_split(
    cs_reviews_df, #<- the full dataset
    test_size=0.2, #<- 20% random sample selected for gold examples
    #cs_examples_df, cs_gold_examples_df=train_test_split(cs_examples_df, test_size=0.2)
    random_state=42 #<- ensures that the splits are the same for every session
)

In [ ]:
(cs_reviews_df.shape, cs_gold_examples_df.shape)

((104, 3), (27, 3))

To select gold examples for this session, we sample randomly from the test data using a `random_state=42`. This ensures that the examples from multiple runs of the sampling are the same (i.e., they are randomly selected but do not change between different runs of the notebook). Note that we are doing this only to keep execution times low for illustration. In practise, large number of gold examples facilitate robust estimates of model accuracy.

In [ ]:
columns_to_select = ['review','sentiment']

In [ ]:
gold_examples = (
        cs_gold_examples_df.loc[:, columns_to_select]
                                     .sample(21, random_state=42) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')
)

In [ ]:
gold_examples

'[{"review":"The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive\'s efforts. The packaging seemed more than adequate to protect the contents during transit.","sentiment":"Positive"},{"review":"ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.","sentiment":"Negative"},{"review":"ExpressWay Logistics\' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their fa

In [ ]:
json.loads(gold_examples)[0]     #Json format

{'review': "The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.",
 'sentiment': 'Positive'}

##**Step 3: Derive Prompt (12 Marks)**

(A) Write Zero Shot System Message (3 Marks)

(B) Create Zero Shot Prompt (2 Marks)

(C) Write Few Shot System Message (3 Marks)

(D) Create Examples For Few shot prompte (2 Marks)

(E) Create Few Shot Prompt (2 Marks)

In [ ]:
user_message_template = "```{courier_service_review}```"
courier_service_review ="""
ExpressWay Logistics is a lifesaver when it comes to shipping. Their prompt and professional service ensures that my packages always arrive on time and in excellent condition.
What sets them apart is their attention to detail - they take the time to understand my specific needs and tailor their solutions accordingly.
I highly recommend ExpressWay Logistics to anyone in need of reliable shipping services."""

**(A) Write Zero Shot System Message (3 Marks)**

In [ ]:
# Write zero shot system message here
zero_shot_system_message = """Classify courier service reviews in the input as positive or negative in sentiment.
Reviews will be delimited by triple backticks, that is, ```.
Do not explain your answer. Your answer should only contain the label: positive or negative.
"""

**(B) Create Zero Shot Prompt (2 Marks)**

In [ ]:
# Create zero shot prompt to be input ready for completion function
zero_shot_prompt = [
    {"role": "system", "review": zero_shot_system_message},
    {"role": "user", "customer_review": user_message_template.format(courier_service_review=courier_service_review)}
]

In [ ]:
num_tokens_from_messages(zero_shot_prompt)

134

**(C) Write Few Shot System Message (3 Marks)**

In [ ]:
few_shot_system_message = """
Extract sentiments from customer reviews in the input.
Reviews will be delimited by triple backticks, that is, ```.
Do not explain your answer.
"""

Merely selecting random samples from the polarity subsets is not enough because the examples included in a prompt are prone to a set of known biases such as:
 - Majority label bias (frequent answers in predictions)
 - Recency bias (examples near the end of the prompt)


To avoid these biases, it is important to have a balanced set of examples that are arranged in random order. Let us create a Python function that generates bias-free examples:

In [ ]:
def create_examples(dataset, n=4):

    """
    Return a JSON list of randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (review + label)
        n (int): number of examples of each class to be selected

    Output:
        randomized_examples (JSON): A JSON with examples in random order
    """

    positive_reviews = (dataset.sentiment == 'Positive')
    negative_reviews = (dataset.sentiment == 'Negative')
    columns_to_select = ['review', 'sentiment']

    positive_examples = dataset.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = dataset.loc[negative_reviews, columns_to_select].sample(n)

    examples = pd.concat([positive_examples, negative_examples])

    # sampling without replacement is equivalent to random shuffling

    randomized_examples = examples.sample(2*n, replace=False)

    return randomized_examples.to_json(orient='records')

**(D) Create Examples For Few shot prompte (2 Marks)**

In [ ]:
# Create Examples
examples = '[{"review" : "The delivery was prompt and the package arrived in perfect condition." ,"sentiment": "Positive"},{ "review" : "I had to wait a long time for my order, an when it finally arrived, the product was damaged and not intact and couriers service unable to manage simultaneous customer requests" , "sentiment": "Negative"}]'

In [ ]:
json.loads(examples)

[{'review': 'The delivery was prompt and the package arrived in perfect condition.',
  'sentiment': 'Positive'},
 {'review': 'I had to wait a long time for my order, an when it finally arrived, the product was damaged and not intact and couriers service unable to manage simultaneous customer requests',
  'sentiment': 'Negative'}]

With the examples in place, we can now assemble a few-shot prompt. Since we will be using the few-shot prompt several times during evaluation, let us write a function to create a few-shot prompt (the logic of this function is depicted below).

In [ ]:
def create_prompt(system_message, examples, user_message_template):

    """
    Return a prompt message in the format expected by the Open AI API.
    Loop through the examples and parse them as user message and assistant
    message.

    Args:
        system_message (str): system message with instructions for sentiment analysis
        examples (str): JSON string with list of examples
        user_message_template (str): string with a placeholder for courier service reviews

    Output:
        few_shot_prompt (List): A list of dictionaries in the Open AI prompt format
    """

    few_shot_prompt = [{'role':'system', 'content': system_message}]

    for example in json.loads(examples):
        example_review = example['review']
        example_sentiment = example['sentiment']

        few_shot_prompt.append(
            {
                'role': 'user',
                'content': user_message_template.format(
                    courier_service_review=example_review
                )
            }
        )

        few_shot_prompt.append(
            {'role': 'assistant', 'content': f"{example_sentiment}"}
        )

    return few_shot_prompt

**(E) Create Few Shot Prompt (2 Marks)**

In [ ]:
# Create Few shot prompt
few_shot_prompt = json.loads(examples) + [
    {"role": "user", "content": user_message_template.format(courier_service_review=courier_service_review)}]

In [ ]:
few_shot_prompt

[{'review': 'The delivery was prompt and the package arrived in perfect condition.',
  'sentiment': 'Positive'},
 {'review': 'I had to wait a long time for my order, an when it finally arrived, the product was damaged and not intact and couriers service unable to manage simultaneous customer requests',
  'sentiment': 'Negative'},
 {'role': 'user',
  'content': '```\nExpressWay Logistics is a lifesaver when it comes to shipping. Their prompt and professional service ensures that my packages always arrive on time and in excellent condition. \nWhat sets them apart is their attention to detail - they take the time to understand my specific needs and tailor their solutions accordingly. \nI highly recommend ExpressWay Logistics to anyone in need of reliable shipping services.```'}]

In [ ]:
num_tokens_from_messages(few_shot_prompt)

137

##**Step 4: Evaluate prompts (8 Marks)**

(A) Evaluate Zero Shot Prompt (2 Marks)

(B) Evaluate Few Shot Prompt (2 marks)

(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)

Now we have two sets of prompts that we need to evaluate using gold labels. Since the few-shot prompt depends on the sample of examples that was drawn to make up the prompt, we expect some variability in evaluation. Hence, we evaluate each prompt multiple times to get a sense of the average and the variation around the average.

To reiterate, a choice on the prompt should account for variability due to the choice of the random sample. To aid repeated evaluation, we assemble an evaluation function .

In [ ]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for courier service review

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths, review_texts = [], [], []

    for example in json.loads(gold_examples):
        gold_input = example['review']
        user_input = [
            {
                'role':'user',
                'content': user_message_template.format(courier_service_review=gold_input)
            }
        ]

        try:
            response = client.chat.completions.create(
                model=chat_model_id,
                messages=prompt+user_input,
                temperature=0, # <- Note the low temperature (For a deterministic response)
                max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            )

            prediction = response.choices[0].message.content
            # response = openai.ChatCompletion.create(
            #     deployment_id=chat_model_id,
            #     messages=prompt+user_input,
            #     temperature=0, # <- Note the low temperature(For a deterministic response)
            #     # max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            # )

            # prediction = response['choices'][0]['message']['content']
            model_predictions.append(prediction.strip()) # <- removes extraneous white spaces
            ground_truths.append(example['sentiment'])
            review_texts.append(gold_input)

        except Exception as e:
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    table_data = [[text, pred, truth] for text, pred, truth in zip(review_texts, model_predictions, ground_truths)]
    headers = ["Review", "Model Prediction", "Ground Truth"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

    return micro_f1_score


Let us now use this function to do one evaluation of all the two prompts assembled so far, each time computing the Micro-F1 score.

**(A) Evaluate zero shot prompt (2 Marks)**

In [ ]:
# 1. Evaluate Zero-shot prompt
print("Evaluating Zero-shot prompt...")
#evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)
#user_message_template.format(courier_service_review=courier_service_review)
zero_shot_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)
print(f"Zero-Shot Micro-F1 Score: {zero_shot_f1:.4f}")

Evaluating Zero-shot prompt...
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                       

**(B) Evaluate few shot prompt (2 Marks)**

In [ ]:
# 2. Evaluate Few-shot prompt
print("Evaluating Few-shot prompt...")
few_shot_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)
print(f"Few-shot Micro-F1 Score: {few_shot_f1}\n")

Evaluating Few-shot prompt...
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                        

However, this is just *one* choice of examples. We will need to run these evaluations with multiple choices of examples to get a sense of variability in F1 score for the few-shot prompt. As an example, let us run evaluations for the few-shot prompt 5 times.

In [ ]:
num_eval_runs = 5

In [ ]:
zero_shot_performance = []
few_shot_performance = []

In [ ]:
for _ in tqdm(range(num_eval_runs)):

    # For each run create a new sample of examples
    examples = create_examples(cs_gold_examples_df)

    # Assemble the zero shot prompt with these examples
    zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]
    # zero_shot_prompt = create_prompt(zero_shot_system_message, examples, user_message_template)

    # Assemble the few shot prompt with these examples
    few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)

    # Evaluate zero shot prompt accuracy on gold examples
    zero_shot_micro_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

    # Evaluate few shot prompt accuracy on gold examples
    few_shot_micro_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

    zero_shot_performance.append(zero_shot_micro_f1)
    few_shot_performance.append(few_shot_micro_f1)

  0%|          | 0/5 [00:00<?, ?it/s]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 20%|██        | 1/5 [00:06<00:25,  6.31s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 40%|████      | 2/5 [00:12<00:18,  6.16s/it]

...

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 80%|████████  | 4/5 [00:25<00:06,  6.29s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

100%|██████████| 5/5 [00:31<00:00,  6.29s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

**(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)**

Compute the average (mean) and measure the variability (standard deviation) of the evaluation scores for both zero shot and few shot prompts.

In [ ]:
# Calculate for Zero Shot
zero_shot_mean = np.mean(zero_shot_performance)
zero_shot_std = np.std(zero_shot_performance)

# Print the results
print(f"Zero-shot Mean Micro-F1: {zero_shot_mean:.4f}, Std: {zero_shot_std:.4f}")


Zero-shot Mean Micro-F1: 0.0476, Std: 0.0000


In [ ]:
# Calculate for Few Shot
few_shot_mean = np.mean(few_shot_performance)
few_shot_std = np.std(few_shot_performance)
print(f"Few-shot Mean Micro-F1: {few_shot_mean:.4f}, Std: {few_shot_std:.4f}")

Few-shot Mean Micro-F1: 0.8381, Std: 0.0233


##**Step 5: Observation and Insights and Business perspective (3 Marks)**

( Based on the projects, learner needs to share observations, learnings, insights and the business use case where these learnings can be beneficial.
Provide a breakdown of the percentage of positive and negative reviews. Additionally, explain how this classification can assist ExpressWay Logistics in addressing the issues identified. )



The evaluation results indicate that the Few-shot prompt significantly outperforms the Zero-shot prompt in terms of predicting sentiment in courier service reviews.

Insights from the Results:
Zero-shot Mean Micro-F1: 0.0476, Std: 0.0000:

The Zero-shot performance is very low (4.76% accuracy), with no variation in the results. This suggests that when the model is not given enough examples of how to classify the reviews (sentiment), it struggles to understand or predict the sentiment correctly.

Conclusion: The model does not have enough context or guidance from the Zero-shot prompt, resulting in poor performance. This could reflect the complexity of the sentiment analysis task, requiring more than just a general understanding of the text.

Few-shot Mean Micro-F1: 0.8381, Std: 0.0233:

The Few-shot performance is much higher (83.81% accuracy), with a small standard deviation (2.33%). This means that when the model is given examples of how to classify courier service reviews, it performs much better and is more consistent.
Conclusion: The model clearly benefits from seeing a few labeled examples, allowing it to learn the structure of the reviews and how to map them to the correct sentiment label. The small standard deviation indicates that the model performs reliably across different evaluation runs.

What It Tells About the Courier Service Reviews:
Complexity of the Reviews: The reviews likely contain nuanced language that requires more contextual understanding to correctly classify the sentiment. Simple Zero-shot approaches (without any examples) fail to capture these nuances, suggesting that these reviews are not trivial to classify.

The large gap between Zero-shot and Few-shot performance indicates that the model needs examples to understand the sentiment patterns in the reviews. This suggests that there may be non-standard, informal, or mixed feedback within the reviews that a more general model can't easily interpret.

Recommendations for Improving Courier Service:
Focus on Consistent and Structured Feedback:

Encourage customers to provide structured feedback by asking for specific aspects of the service (e.g., delivery speed, package handling, customer service). This would make it easier to identify patterns in reviews and address key areas for improvement.
Identify Common Themes in Negative Reviews:

With the high performance of the Few-shot model, we can now better categorize the negative reviews. Focus on identifying recurring complaints (e.g., delays, lost packages, customer support issues) and address them systematically.
If we notice frequent patterns of dissatisfaction in a particular area (e.g., delivery speed), we can target operational improvements more effectively.

Improve Service Based on Positive and Negative Sentiment:

By analyzing which areas of your service receive positive feedback and which areas are often criticized, we can focus on reinforcing strengths (e.g., timely deliveries, good customer support) while addressing weaknesses (e.g., handling of fragile packages).

Leverage Customer Feedback for Training:
Consider incorporating a larger dataset of labeled reviews into our sentiment analysis model for continued training, which will help the model generalize better and capture even more subtle patterns in customer sentiment over time.
By continuously monitoring sentiment in reviews using a Few-shot prompt, ExpressWay Logistics can prioritize customer service improvements, identify specific problem areas, and respond proactively to customer needs.

In [ ]:
!pip install nbconvert

In [ ]:
!jupyter nbconvert --to html /content/drive/MyDrive/Colab_Notebooks/GEN-AI-Final-Project/AHS_Learner_Notebook_final_project_GEN_AI_MSFT.ipynb

[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab_Notebooks/GEN-AI-Final-Project/AHS_Learner_Notebook_final_project_GEN_AI_MSFT.ipynb to html
[NbConvertApp] Writing 1031189 bytes to /content/drive/MyDrive/Colab_Notebooks/GEN-AI-Final-Project/AHS_Learner_Notebook_final_project_GEN_AI_MSFT.html
